# ACM AI | Beginner Track Bonus Material: Intro to Tensorflow
Tensorflow is the framework of choice for many in the Machine Learning field. This intro should help you get started with using Tensorflow.

## What is a tensor?

A tensor is an n-dimensional array. 

t1 = [1,2,3,4,5] is a 1-dimesional array,

t2 = [
[1,2,3,12],
[4,5,6,45],
[7,8,9,78]
] is a 2-dimensional array, ... etc.

### Rank of a tensor
A tensor's rank is the **number of dimensions** the tensor has. It is basically ***n***, in the above definition

### Shape of a tensor
The shape of a tensor is a **list specifying the number of items in each dimension**. In the above example, the shape of t1 is [5], while the shape of t2 is [3,4]. 

How is the size of the shape (yup, that makes sense) related to the rank?

## How Tensorflow works

Tensorflow uses a dataflow graph to represet a program. Every **edge** in the graph is a **Tensor**, every **node** is an **Operation**, and the Tensors "**flow**" from one Operation to the next. 

![](https://www.tensorflow.org/images/tensors_flowing.gif)

When you're writing a Tensorflow program, you're building this graph. But the Tensors in the graph (the edges) do not store the actual results of the Operations. The Tensors are just **handles**, telling the graph that a result is expected there. To actually get some actual results, you need to create a **Session** and **run** it. 

Let's get started

In [0]:
import tensorflow as tf
import numpy

In [31]:
#Let's define some constants.
c1 = tf.constant(3, dtype=tf.int32, shape = ()) # creates a tf.Operation that always produces the value 3. Returns a Tensor
c2 = tf.constant(14, dtype=tf.int32, shape = ()) # you can also create arrays by changing the shape parameter

#Let's multiply them
mult = tf.multiply(c1,c2) # c1 * c2 also works, * is overloaded

#Create a session
with tf.Session() as sess:
  print(sess.run(mult)) #To get the value of a Tensor or Operation, pass it into sess.run()

42


What if we don't know what values some input is going to take while writing the program.

We can create a **Placeholder**

In [0]:
p1 = tf.placeholder(tf.int32, shape=(2,3))
p2 = tf.placeholder(tf.int32, shape=(2,3))

add = p1 + p2 #why did we not have to use tf.add()? (Hint: overloading)

arr1 = numpy.ndarray((2,3))
arr2 = numpy.ndarray((2,3))
arr3 = numpy.ndarray((2,3))

#creating some arbitrary arrays
for i in range(2):
  for j in range(3):
    arr1[i,j] = i + j
    arr2[i,j] = i * j
    arr3[i,j] = i ** j

In [33]:
with tf.Session() as sess:
  print(sess.run(add, feed_dict={p1:arr1, p2:arr2})) # need to specify the value for the placeholders while running

[[0 1 2]
 [1 3 5]]


In [34]:
with tf.Session() as sess:
  print(sess.run(add, feed_dict={p1:arr1, p2:arr3})) # with different p2

[[1 1 2]
 [2 3 4]]


Now what if we want to store data that is maintained across multiple run()s? We use **Variables**

Why would we need to maintain such data? 

In [0]:
var1 = tf.Variable([0,0,0]) # shape and data type of variable is determined by the initial value provided as argument

toAdd = tf.constant([1,1,1], dtype=tf.int32)

assign_op = tf.assign(var1, var1 + toAdd)

init_ = tf.global_variables_initializer() #before a variable can be used for the first time, it must be initialized

In [36]:
with tf.Session() as sess:
  sess.run(init_)
  for i in range(5):
    print(sess.run(assign_op))
    
#We see that the value of var1 persists across the multiple runs: it doesn't start off from [0,0,0] every time
#WARNING: We've used tf.assign() here, but it is slightly more complicated than it seems. 
#Things start getting weird when other operations depend on var1. You might expect the assign_op to run before, but it ends up not running
#We probably won't be using tf.assign() for anything other than explanatory purposes
#Come up and talk to us later if you're curious about this

[1 1 1]
[2 2 2]
[3 3 3]
[4 4 4]
[5 5 5]


So let's try finding the minimum of a function using Tensorflow. 
Tensorflow provides some nice tools called **Optimizers** that can do just this. 

Let's see how this is done

In [0]:
tf.reset_default_graph()    #Let's clean up our graph

#get_variable is another way (the one we'd recommend) for defining variables in Tensorflow
#While using this you are forced to give the variable a name
#Keeping track of variables, thus, becomes easier, and a lot more functionality is added
#WARNING: You can't use get_variable to create two variables of the same name. 
#Things, again, get complicated because of the way Tensorflow works
#You'll have to know about collections, etc., so come talk to us to know more


x = tf.get_variable("indep_var", dtype=tf.float32, initializer=tf.constant(100.0)) #try changing the inital value of the variable to see what happens

f = tf.multiply(x,x)

#This is one sort of Optimizer: Gradient Descet Optimizer
grad = tf.train.GradientDescentOptimizer(learning_rate = 0.1) #what happens if we use learning_rate = 1? Why?

#We want to minimize f
trainer = grad.minimize(f)

init_ = tf.global_variables_initializer()

In [38]:
num_epochs = 100

with tf.Session() as sess:
  sess.run(init_)
  for i in range(num_epochs):
    _, xnew = sess.run([trainer, x])
    print(xnew)

80.0
64.0
51.2
40.96
32.767998
26.214397
20.971518
16.777214
13.421771
10.737417
8.589933
6.871947
5.4975576
4.398046
3.518437
2.8147495
2.2517996
1.8014396
1.4411517
1.1529214
0.9223372
0.73786974
0.5902958
0.47223663
0.37778932
0.30223146
0.24178517
0.19342813
0.15474251
0.123794004
0.0990352
0.07922816
0.06338253
0.05070602
0.040564816
0.032451853
0.025961483
0.020769186
0.01661535
0.01329228
0.010633824
0.00850706
0.006805648
0.0054445183
0.0043556145
0.0034844917
0.0027875933
0.0022300747
0.0017840598
0.0014272479
0.0011417983
0.0009134387
0.00073075097
0.0005846008
0.00046768063
0.0003741445
0.0002993156
0.00023945249
0.000191562
0.0001532496
0.00012259968
9.807975e-05
7.8463796e-05
6.277104e-05
5.0216833e-05
4.0173465e-05
3.213877e-05
2.5711017e-05
2.0568814e-05
1.6455051e-05
1.3164041e-05
1.0531233e-05
8.424986e-06
6.7399887e-06
5.391991e-06
4.313593e-06
3.4508744e-06
2.7606995e-06
2.2085596e-06
1.7668477e-06
1.4134782e-06
1.1307826e-06
9.0462606e-07
7.2370085e-07
5.789607e-07


Let's try something harder. Say we want to minimize

$f(x,y) = 3x^2 + 2y^2 + 3x + y + 17$

That is, we want to find that $x$ and $y$ for which $f(x,y)$ is minimum

How would we do this? 

In [0]:
tf.reset_default_graph()

#Write code below here
x = tf.get_variable("indep_var", dtype=tf.float32, initializer=tf.constant(220.0))
y = tf.get_variable("indep_var2", dtype=tf.float32, initializer=tf.constant(76.0))

f = 3*tf.multiply(x,x) + 2*tf.multiply(y,y) + 3*x + y + 17
grad = tf.train.GradientDescentOptimizer(learning_rate = 0.05)
trainer = grad.minimize(f)

init_ = tf.global_variables_initializer()


In [56]:
num_epochs = 100

with tf.Session() as sess:
  sess.run(init_)
  for i in range(num_epochs):
    _, xnew, ynew = sess.run([trainer, x, y])
    print("x: " + str(xnew) + "y:" + str(ynew))

x: 153.85y:60.75
x: 107.545006y:48.55
x: 75.13151y:38.79
x: 52.442055y:30.982
x: 36.559437y:24.7356
x: 25.441605y:19.73848
x: 17.659122y:15.740784
x: 12.211386y:12.542627
x: 8.39797y:9.984102
x: 5.728579y:7.9372816
x: 3.8600051y:6.299825
x: 2.5520036y:4.98986
x: 1.6364025y:3.941888
x: 0.9954817y:3.1035106
x: 0.54683715y:2.4328084
x: 0.232786y:1.8962467
x: 0.01295019y:1.4669974
x: -0.14093487y:1.1235979
x: -0.24865441y:0.84887826
x: -0.3240581y:0.6291026
x: -0.37684065y:0.45328206
x: -0.41378844y:0.31262565
x: -0.4396519y:0.20010051
x: -0.45775634y:0.1100804
x: -0.47042942y:0.038064312
x: -0.4793006y:-0.019548548
x: -0.4855104y:-0.06563884
x: -0.4898573y:-0.10251107
x: -0.4929001y:-0.13200885
x: -0.49503008y:-0.15560707
x: -0.49652106y:-0.17448565
x: -0.49756473y:-0.18958852
x: -0.4982953y:-0.20167081
x: -0.49880672y:-0.21133664
x: -0.4991647y:-0.21906932
x: -0.49941528y:-0.22525546
x: -0.4995907y:-0.23020437
x: -0.49971348y:-0.2341635
x: -0.49979943y:-0.2373308
x: -0.4998596y:-0.239864